In [ ]:
! pwd && ls

/content
sample_data


In [ ]:
! wget https://speed.hetzner.de/10GB.bin

--2021-05-23 11:13:31--  https://speed.hetzner.de/10GB.bin
Resolving speed.hetzner.de (speed.hetzner.de)... 88.198.248.254, 2a01:4f8:0:59ed::2
Connecting to speed.hetzner.de (speed.hetzner.de)|88.198.248.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10485760000 (9.8G) [application/octet-stream]
Saving to: ‘10GB.bin’

10GB.bin            100%[===================>]   9.77G  28.3MB/s    in 7m 19s  

2021-05-23 11:20:50 (22.8 MB/s) - ‘10GB.bin’ saved [10485760000/10485760000]



In [ ]:
! wget http://speedtest.belwue.net/100G

--2021-05-23 11:24:47--  http://speedtest.belwue.net/100G
Resolving speedtest.belwue.net (speedtest.belwue.net)... 129.143.4.238, 2001:7c0:0:1002::b
Connecting to speedtest.belwue.net (speedtest.belwue.net)|129.143.4.238|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107374182400 (100G)
Saving to: ‘100G’

100G                 32%[=====>              ]  32.23G  28.3MB/s    eta 40m 11s^C


In [ ]:
! ls

sample_data


## DAtasets

In [ ]:
def print_shapes(path):
  import numpy as np
  from PIL import Image

  print(np.array(Image.open(path)).shape)

## Set5

In [ ]:
! wget https://uofi.box.com/shared/static/kfahv87nfe8ax910l85dksyl2q212voc.zip


--2021-07-16 05:44:56--  https://uofi.box.com/shared/static/kfahv87nfe8ax910l85dksyl2q212voc.zip
Resolving uofi.box.com (uofi.box.com)... 107.152.29.197
Connecting to uofi.box.com (uofi.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/kfahv87nfe8ax910l85dksyl2q212voc.zip [following]
--2021-07-16 05:44:56--  https://uofi.box.com/public/static/kfahv87nfe8ax910l85dksyl2q212voc.zip
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/kfahv87nfe8ax910l85dksyl2q212voc.zip [following]
--2021-07-16 05:44:56--  https://uofi.app.box.com/public/static/kfahv87nfe8ax910l85dksyl2q212voc.zip
Resolving uofi.app.box.com (uofi.app.box.com)... 107.152.26.201
Connecting to uofi.app.box.com (uofi.app.box.com)|107.152.26.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [ ]:
! unzip *.zip

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
hr = '/content/Set5/image_SRF_2/img_001_SRF_2_HR.png'
lr = '/content/Set5/image_SRF_2/img_001_SRF_2_LR.png'

In [ ]:
np.array(Image.open(lr)).shape, np.array(Image.open(hr)).shape

((256, 256, 3), (512, 512, 3))

In [ ]:
! rm *zip

## Set14

In [ ]:
! wget https://uofi.box.com/shared/static/igsnfieh4lz68l926l8xbklwsnnk8we9.zip

--2021-07-16 05:51:34--  https://uofi.box.com/shared/static/igsnfieh4lz68l926l8xbklwsnnk8we9.zip
Resolving uofi.box.com (uofi.box.com)... 107.152.29.197
Connecting to uofi.box.com (uofi.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/igsnfieh4lz68l926l8xbklwsnnk8we9.zip [following]
--2021-07-16 05:51:34--  https://uofi.box.com/public/static/igsnfieh4lz68l926l8xbklwsnnk8we9.zip
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/igsnfieh4lz68l926l8xbklwsnnk8we9.zip [following]
--2021-07-16 05:51:35--  https://uofi.app.box.com/public/static/igsnfieh4lz68l926l8xbklwsnnk8we9.zip
Resolving uofi.app.box.com (uofi.app.box.com)... 107.152.29.201
Connecting to uofi.app.box.com (uofi.app.box.com)|107.152.29.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [ ]:
! unzip *zip

In [ ]:
hr = '/content/Set14/image_SRF_2/img_014_SRF_2_HR.png'
lr = '/content/Set14/image_SRF_2/img_014_SRF_2_LR.png'

In [ ]:
print_shapes(hr), print_shapes(lr)

(390, 586, 3)
(195, 293, 3)


(None, None)

In [ ]:
! rm *zip

## urban100

In [ ]:
! wget https://uofi.box.com/shared/static/65upg43jjd0a4cwsiqgl6o6ixube6klm.zip

--2021-07-16 05:57:18--  https://uofi.box.com/shared/static/65upg43jjd0a4cwsiqgl6o6ixube6klm.zip
Resolving uofi.box.com (uofi.box.com)... 107.152.26.197
Connecting to uofi.box.com (uofi.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/65upg43jjd0a4cwsiqgl6o6ixube6klm.zip [following]
--2021-07-16 05:57:18--  https://uofi.box.com/public/static/65upg43jjd0a4cwsiqgl6o6ixube6klm.zip
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/65upg43jjd0a4cwsiqgl6o6ixube6klm.zip [following]
--2021-07-16 05:57:18--  https://uofi.app.box.com/public/static/65upg43jjd0a4cwsiqgl6o6ixube6klm.zip
Resolving uofi.app.box.com (uofi.app.box.com)... 107.152.29.201
Connecting to uofi.app.box.com (uofi.app.box.com)|107.152.29.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [ ]:
! unzip *zip

In [ ]:
! mkdir urban100 

In [ ]:
! rm *zip

In [ ]:
lr = '/content/urban100/image_SRF_2/img_069_SRF_2_LR.png'
hr = '/content/urban100/image_SRF_2/img_069_SRF_2_HR.png'

In [ ]:
print_shapes(hr), print_shapes(lr)

(412, 600, 3)
(206, 300, 3)


(None, None)

## BSD100

In [ ]:
! wget https://uofi.box.com/shared/static/qgctsplb8txrksm9to9x01zfa4m61ngq.zip

--2021-07-16 06:16:06--  https://uofi.box.com/shared/static/qgctsplb8txrksm9to9x01zfa4m61ngq.zip
Resolving uofi.box.com (uofi.box.com)... 107.152.26.197
Connecting to uofi.box.com (uofi.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/qgctsplb8txrksm9to9x01zfa4m61ngq.zip [following]
--2021-07-16 06:16:07--  https://uofi.box.com/public/static/qgctsplb8txrksm9to9x01zfa4m61ngq.zip
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/qgctsplb8txrksm9to9x01zfa4m61ngq.zip [following]
--2021-07-16 06:16:07--  https://uofi.app.box.com/public/static/qgctsplb8txrksm9to9x01zfa4m61ngq.zip
Resolving uofi.app.box.com (uofi.app.box.com)... 107.152.26.201
Connecting to uofi.app.box.com (uofi.app.box.com)|107.152.26.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [ ]:
! unzip *zip

In [ ]:
! mkdir bsd100

In [ ]:
! rm *zip

## Sun Hays 80

In [ ]:
! wget https://uofi.box.com/shared/static/rirohj4773jl7ef752r330rtqw23djt8.zip

--2021-07-16 06:35:00--  https://uofi.box.com/shared/static/rirohj4773jl7ef752r330rtqw23djt8.zip
Resolving uofi.box.com (uofi.box.com)... 107.152.26.197
Connecting to uofi.box.com (uofi.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/rirohj4773jl7ef752r330rtqw23djt8.zip [following]
--2021-07-16 06:35:00--  https://uofi.box.com/public/static/rirohj4773jl7ef752r330rtqw23djt8.zip
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/rirohj4773jl7ef752r330rtqw23djt8.zip [following]
--2021-07-16 06:35:00--  https://uofi.app.box.com/public/static/rirohj4773jl7ef752r330rtqw23djt8.zip
Resolving uofi.app.box.com (uofi.app.box.com)... 107.152.26.201
Connecting to uofi.app.box.com (uofi.app.box.com)|107.152.26.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [ ]:
! unzip -q *zip

In [ ]:
! mkdir sunhays80

In [ ]:
! rm *zip

In [ ]:
while True:
  pass

# bicubic downsampling matlab

In [ ]:
%%capture
! apt install octave

In [ ]:
! touch temp.m

In [ ]:
! sudo apt-get install octave-image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  octave-image
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 503 kB of archives.
After this operation, 2,874 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 octave-image amd64 2.6.2-5 [503 kB]
Fetched 503 kB in 2s (260 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package octave-image.
(Reading database ... 168577 files and direct

In [ ]:
! octave -W temp.m

Image taken


## Stacked attention Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
	def __init__(self, hidden_dim, img_dim, ques_dim):
		super(Attention, self).__init__()
		self.w1 = nn.Linear(img_dim, hidden_dim, bias=False)
		self.w2 = nn.Linear(ques_dim, hidden_dim)
		self.wp = nn.Linear(hidden_dim, 1)
		self.tanh = torch.tanh
	
	def forward(self, vi, vq):
		"""
		Summary: 
			d = img_dim = ques_dim
		Args:
			vi (m, d): Visual features of the image, where m = (width * height) of feature map
			vq (d):    Features of the question

		Returns:
			pi (m): Attention coeeficient (alpha) corresponding to each feature vector
		"""
		# h.shape: (hidden_dim, m)
		a = self.w1(vi)
		b = torch.unsqueeze(self.w2(vq), -2)
		print(f'A: {a.shape}, B: {b.shape}')
		# a, b = torch.transpose(a, -1, -2), b
		print(f'A: {a.shape}, B: {b.shape}')
		h = self.tanh(a + b)
		print(f'Shape of h: {h.shape}')
		# pi.shape: (m, 1)
		pi = F.softmax(self.wp(h))
		return pi

In [ ]:
class AttentionLayer(nn.Module):
	def __init__(self, hidden_dim, img_dim, ques_dim):
		super(AttentionLayer, self).__init__()
		self.attention = Attention(hidden_dim, img_dim, ques_dim)

	def forward(self, vi, u):
		"""[summary]

		Args:
			vi (m, d): Visual features of image, where m = (width * height) of feature map
			u (d):     Features of the question

		Returns:
			u (d): Features of question after applying the attention over the image
		"""
		pi = self.attention(vi, u)
		u = torch.sum(pi * vi) + torch.unsqueeze(u, dim=-2)
		return torch.squeeze(u, dim=-2)

In [ ]:
m, d, hidden_dim = (14 * 14), 120, 53
bs = 2

vi = torch.rand((bs, m, d))
u = torch.rand((bs, d))

In [ ]:
model = AttentionLayer(hidden_dim, d, d)
out = model(vi, u)
print(out.shape)

A: torch.Size([2, 196, 53]), B: torch.Size([2, 1, 53])
A: torch.Size([2, 196, 53]), B: torch.Size([2, 1, 53])
Shape of h: torch.Size([2, 196, 53])
torch.Size([2, 120])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
_attention_model = Attention(hidden_dim, d, d)
print(f'vi: {vi.shape}, u: {u.shape}')
out = _attention_model(vi, u)
print(out.shape)
assert out.shape == (bs, m, 1)

vi: torch.Size([2, 196, 120]), u: torch.Size([2, 120])
A: torch.Size([2, 196, 53]), B: torch.Size([2, 1, 53])
A: torch.Size([2, 196, 53]), B: torch.Size([2, 1, 53])
Shape of h: torch.Size([2, 196, 53])
torch.Size([2, 196, 1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
a = torch.tensor([[
                [1, 2, 4],
                [3, 4, 5]
]])

b = torch.tensor([[
                  1, 2
]])

print(f'A: {a.shape}, B: {b.shape}, ')

A: torch.Size([1, 2, 3]), B: torch.Size([1, 2]), 


In [ ]:
torch.transpose(torch.transpose(a, -1, -2) + b, -1, -2)

tensor([[[2, 3, 5],
         [5, 6, 7]]])

In [ ]:
torch.add(a.view(1, 3, 2), b).view(1, 2, 3)

tensor([[[2, 4, 5],
         [5, 5, 7]]])

In [ ]:
a = torch.ones((2, 3, 5))
b = torch.ones(2, 5)
b = torch.unsqueeze(b, -2)
a + b

tensor([[[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]]])

In [ ]:
torch.transpose(a, -1, -2).shape

torch.Size([2, 5, 3])

##### RNN Part: LSTM

In [ ]:
import torch
import torch.nn as nn


In [ ]:
vocab_size, embed_size = 500, 250

In [ ]:
class QuestionEncoder(nn.Module):
    def __init__(self, vocab_size, embed_size, out_size):
        super(QuestionEncoder, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.out_size = out_size

        self.embed = nn.Embedding(self.vocab_size, self.embed_size)
        self.lstm = nn.LSTM(self.embed_size, self.out_size, batch_first=True)

    def forward(self, x):
        """Implements the class to encode the questions into feature vectors. Uses embeddings and LSTMs for the same
        
        Args:
            x (N, seq_len): Tensor containing numbers corresponding to words of the sentence.
        
        Returns:
            c (N, out_size): Feature vector corresponding to each question vector
        """
        output, (h, c) = self.lstm(self.embed(x))
        return torch.squeeze(c, dim=0)

In [ ]:
model = QuestionEncoder(vocab_size, embed_size, out_size=90)

In [ ]:
# batch of 2 sentences, each containing 4 words
sentence = torch.tensor([
                         [1, 2, 3, 4],
                         [5, 6, 7, 8]
])

In [ ]:
out = model(sentence)
# sentence.shape, out.shape
assert tuple(out.shape) == (2, 90)

## Image Encoder

In [ ]:
! pip install torchsummary

In [ ]:
from torchvision.models import vgg19
from torchsummary import summary

In [ ]:
class ImageEncoder(nn.Module):
  def __init__(self, out_dim):
    super(ImageEncoder, self).__init__()
    self.out_dim = out_dim
    self.vgg = vgg19(pretrained=True)
    self.vgg.avgpool = nn.Sequential()
    self.vgg.classifier = nn.Sequential()
    self.fc = nn.Linear(512, out_dim)

  def forward(self, x):
    feats = self.vgg(x)
    batch_size = x.shape[0]
    feats = feats.view(batch_size, 14 * 14, 512)
    return self.fc(feats)


In [ ]:
model = ImageEncoder(out_dim=250)
inp = torch.rand((1, 3, 448, 448))
out = model(inp)
out.shape

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


torch.Size([1, 196, 250])

In [ ]:
summary(vgg19(), input_size=(3, 448, 448))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 448, 448]           1,792
              ReLU-2         [-1, 64, 448, 448]               0
            Conv2d-3         [-1, 64, 448, 448]          36,928
              ReLU-4         [-1, 64, 448, 448]               0
         MaxPool2d-5         [-1, 64, 224, 224]               0
            Conv2d-6        [-1, 128, 224, 224]          73,856
              ReLU-7        [-1, 128, 224, 224]               0
            Conv2d-8        [-1, 128, 224, 224]         147,584
              ReLU-9        [-1, 128, 224, 224]               0
        MaxPool2d-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 256, 112, 112]         295,168
             ReLU-12        [-1, 256, 112, 112]               0
           Conv2d-13        [-1, 256, 112, 112]         590,080
             ReLU-14        [-1, 256, 1

In [ ]:
vgg19()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [1]:
!pwd && ls

/content
sample_data


# SAN model

In [1]:
%%capture
! pip install torchinfo

In [2]:
%%capture
! wget https://raw.githubusercontent.com/harshraj22/DeepLearning/main/implementations/SAN/models/attention.py
! wget https://raw.githubusercontent.com/harshraj22/DeepLearning/main/implementations/SAN/models/image_encoder.py
! wget https://raw.githubusercontent.com/harshraj22/DeepLearning/main/implementations/SAN/models/question_encoder.py
! wget https://raw.githubusercontent.com/harshraj22/DeepLearning/main/implementations/SAN/models/san.py
! wget https://raw.githubusercontent.com/harshraj22/DeepLearning/main/implementations/SAN/utils/dataset.py

In [6]:
import gc
gc.collect()

298

In [3]:
import torch
import torch.nn as nn

# from attention import AttentionLayer
from image_encoder import ImageEncoder
# from question_encoder import QuestionEncoder
# from san import SAN

import warnings
warnings.filterwarnings("ignore")

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms


class QuestionEncoder(nn.Module):
    def __init__(self, vocab_size, embed_size, out_size):
        super(QuestionEncoder, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.out_size = out_size

        self.embed = nn.Embedding(self.vocab_size, self.embed_size)
        self.lstm = nn.LSTM(self.embed_size, self.out_size, batch_first=True)

    def forward(self, x):
        """Implements the class to encode the questions into feature vectors. Uses embeddings and LSTMs for the same
        
        Args:
            x (N, seq_len): Tensor containing numbers corresponding to words of the sentence.
        
        Returns:
            c (N, out_size): Feature vector corresponding to each question vector
        """
        output, (h, c) = self.lstm(self.embed(x))
        return torch.squeeze(h, dim=0)


if __name__ == '__main__':
    vocab_size, embed_size, out_size = 500, 250, 90
    model = QuestionEncoder(vocab_size, embed_size, out_size)
    # batch of 2 sentences, each containing 4 words
    sentence = torch.tensor([
                             [1, 2, 3, 4],
                             [5, 6, 7, 8]
    ])
    out = model(sentence)
    assert tuple(out.shape) == (2, out_size)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
	def __init__(self, hidden_dim, img_dim, ques_dim):
		super(Attention, self).__init__()
		self.w1 = nn.Linear(img_dim, hidden_dim, bias=False)
		self.w2 = nn.Linear(ques_dim, hidden_dim)
		self.wp = nn.Linear(hidden_dim, 1)
		self.tanh = torch.tanh
	
	def forward(self, vi, vq):
		"""
		Summary: 
			d = img_dim = ques_dim
		Args:
			vi (m, d): Visual features of the image, where m = (width * height) of feature map
			vq (d):    Features of the question

		Returns:
			pi (m): Attention coeeficient (alpha) corresponding to each feature vector
		"""
		assert vi.shape[-1] == vq.shape[-1], "Dimentions of Vi and Vq don't match during Attention"
		a = self.w1(vi)
		# vq = vq / 800
		# norm = transforms.Normalize((0.5, 0.5), (0.5, 0.5))
		# vq = F.normalize(vq)
		# unsqueeze for adding vector to matrix.
		# read about broadcasting: https://pytorch.org/docs/stable/notes/broadcasting.html
		b = torch.unsqueeze(self.w2(vq), -2)
		# h.shape: (m, hidden_dim)
		h = self.tanh(a + b)
		# print(f'Attention: a: {a},\n vq: {vq} b: {b},\n h: {h}, wp(h): {self.wp(h)}')
		# pi.shape: (m, 1)
		pi = F.softmax(self.wp(h), dim=1)
        # print(f'Pi: {pi}')
		return pi
	
	
class AttentionLayer(nn.Module):
	def __init__(self, hidden_dim, img_dim, ques_dim):
		super(AttentionLayer, self).__init__()
		self.attention = Attention(hidden_dim, img_dim, ques_dim)

	def forward(self, vi, u):
		"""Implements Stacked Attention Layer. Question feature 'u' is repeatedly passed through this layer,
		it calculates the attention over the image features, and merges the features into itself.

		Args:
			vi (m, d): Visual features of image, where m = (width * height) of feature map
			u (d):     Features of the question

		Returns:
			u (d): Features of question after applying the attention over the image
		"""
		pi = self.attention(vi, u)
		# assert pi.shape[1] == vi.shape[1], f"pi ({pi.shape}) and vi ({vi.shape}) don't have same shape in AttentionLayer"
		u = torch.sum(pi * vi) + torch.unsqueeze(u, dim=-2)

		# print(f'pi:{pi.shape} {pi}\nU: {u.shape} {u}')
		return torch.squeeze(u, dim=-2)
	
	
# if __name__ == '__main__':
# 	m, d, hidden_dim = (14 * 14), 120, 53
# 	bs = 2

# 	vi = torch.rand((bs, m, d))
# 	u = torch.rand((bs, d))

# 	model = Attention(hidden_dim, d, d)
# 	out = model(vi, u)

# 	assert out.shape == (bs, m, 1)


In [6]:
import torch
import torch.nn as nn

# from attention import AttentionLayer
from image_encoder import ImageEncoder
from question_encoder import QuestionEncoder



class SAN(nn.Module):
  def __init__(self, feat_dim, vocab_size, embed_size, n_attention_stacks, hidden_dim_img):
    super(SAN, self).__init__()
    self.feat_dim = feat_dim
    self.vocab_size = vocab_size
    self.embed_size = embed_size

    self.img_enc = ImageEncoder(feat_dim)
    self.ques_enc = QuestionEncoder(vocab_size, embed_size, feat_dim)
    self.att = nn.ModuleList([
              AttentionLayer(hidden_dim_img, feat_dim, feat_dim) for _ in range(n_attention_stacks)
    ])

    self.pred = nn.Linear(feat_dim, vocab_size)


  def forward(self, img, ques):
    """Returns one word output corresponding to input image and question
    
    Args:
        img (N, 3, 448, 448): Image of given shape, constrained due to use of vgg
        ques (N, seq_len): Question encoded as integers
    
    Returns:
        u (N, vocab_size): Vectors of vocab_size, for picking the most probable word
    """

    img = self.img_enc(img)
    img = F.normalize(img)
    ques = self.ques_enc(ques)
    ques = F.normalize(ques)
    
    u = ques
    for layer in self.att:
      u = layer(img, u)
      u = F.normalize(u)
    return self.pred(u)



# if __name__ == '__main__':
#     vocab_size = 500
#     seq_len = 12
#     feat_dim = 600
#     embed_size = 250
#     n_attention_stacks = 5
#     hidden_dim_img = 32

#     model = SAN(feat_dim, vocab_size, embed_size, n_attention_stacks, hidden_dim_img)
#     img = torch.rand((1, 3, 448, 448))
#     ques = torch.randint(0, vocab_size, (1, seq_len))
#     out = model(img, ques)
#     assert tuple(out.shape) == (1, vocab_size)




In [7]:
vocab_size = 9000 # from bert
seq_len = 12 # from dataset
feat_dim = 640 # from paper, the final vector vq, vi
embed_size = 500 # from paper, dimention of embedding of each word
n_attention_stacks = 5
hidden_dim_img = feat_dim
batch_size = 10

In [45]:
model = SAN(feat_dim, vocab_size, embed_size, n_attention_stacks, hidden_dim_img)
img = torch.rand((batch_size, 3, 448, 448))
img = torch.randint(0, 255, (batch_size, 3, 448, 448)).float()
ques = torch.randint(0, vocab_size, (batch_size, seq_len), dtype=torch.int32)
out = model(img, ques)
assert tuple(out.shape) == (batch_size, vocab_size)

pi:torch.Size([10, 196, 1]) tensor([[[0.0051],
         [0.0054],
         [0.0051],
         ...,
         [0.0051],
         [0.0051],
         [0.0051]],

        [[0.0051],
         [0.0053],
         [0.0051],
         ...,
         [0.0051],
         [0.0051],
         [0.0051]],

        [[0.0051],
         [0.0055],
         [0.0051],
         ...,
         [0.0051],
         [0.0051],
         [0.0051]],

        ...,

        [[0.0051],
         [0.0053],
         [0.0051],
         ...,
         [0.0051],
         [0.0051],
         [0.0051]],

        [[0.0051],
         [0.0054],
         [0.0051],
         ...,
         [0.0051],
         [0.0051],
         [0.0051]],

        [[0.0051],
         [0.0054],
         [0.0051],
         ...,
         [0.0051],
         [0.0051],
         [0.0051]]], grad_fn=<SoftmaxBackward>)
U: torch.Size([10, 1, 640]) tensor([[[-0.1031, -0.0774, -0.0837,  ..., -0.1456, -0.0355, -0.1046]],

        [[-0.1393, -0.0134, -0.0439,  ..., -0.0801

In [ ]:
# from torchinfo import summary
# summary(model, input_size=[(1, 3, 448, 448), (1, seq_len,)], input_data=[torch.float32, torch.float32])
# summary(model, input_data=[img, ques])

In [8]:
%%capture
! pip install transformers

In [9]:
from transformers import BertTokenizer, DistilBertTokenizer, BertTokenizerFast

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = DistilBertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

<hr>

In [ ]:
# from transformers import XLNetTokenizer
# tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

In [ ]:
tokenizer.encode('He is good')

[101, 2002, 2003, 2204, 102]

In [ ]:
tokenizer.decode([101, 2002, 2003, 2204, 102])

'[CLS] he is good [SEP]'

In [ ]:
tokenizer.tokenize('He isnt good')

['he', 'isn', '##t', 'good']

In [40]:
tokenizer.encode('desk')

[101, 4624, 102]

In [41]:
tokenizer.convert_tokens_to_ids(['desk'])

[4624]

In [ ]:
tokenizer.convert_ids_to_tokens([2002, 2003, 2204, 100])

['he', 'is', 'good', '[UNK]']

In [ ]:
tokenizer.vocab_size

30522

In [ ]:
tokenizer.convert_ids_to_tokens([0, 1, 2, 1200, 30520, 9011])

['[PAD]', '[unused0]', '[unused1]', 'х', '##？', 'manila']

In [ ]:
tokenizer.convert_tokens_to_ids(['<UNK>', '<STRT>', '<SEP>', '<PAD>', '<END>', '-+i-'])

[100, 100, 100, 100, 100, 100]

In [ ]:
tokenizer.requires_grad

## Create Dataset

In [10]:
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm
from PIL import Image
import numpy as np
from dataset import VQADataset

In [32]:
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm
import torch
from PIL import Image
import numpy as np
from torch.nn.utils.rnn import pad_sequence

"""
Write your own dataset here.
The __getitem__ method must return a dict with the following key, value pairs:
    'ques': tensor of ints, representing the index of words in the vocab
    'ans': tensor of int, representing the index of word answer
    'img': tensor representing the image
    
Get Images for the dataset:
    ! wget http://datasets.d2.mpi-inf.mpg.de/mateusz14visual-turing/nyu_depth_images.tar
Get Train question & ans:
    ! wget https://raw.githubusercontent.com/jayantk/lsp/master/data/daquar/reduced/qa.37.raw.train.txt
DAQAR dataset:
    https://github.com/jayantk/lsp/tree/master/data/daquar
"""


class VQADataset(Dataset):
  def __init__(self, ques_file, image_dir, tokenizer, max_len=30):
    super(Dataset, self).__init__()
    self.ques_file = ques_file
    self.img_dir = image_dir
    self.tokenizer = tokenizer
    self.max_sentence_len = max_len

    self.data = []
    self.load_data()

  def load_data(self):
    with open(self.ques_file, 'r') as f:
      data = f.readlines()

    for index, line in tqdm(enumerate(data[::2]), desc='Iterating over questions'):
      img = line.replace('?', '').strip(' ').split()[-1] + '.png'
      ques = [x for x in self.tokenizer.encode(line)]
      ques = [torch.tensor(min(x, vocab_size-1)) for x in ques]

      ans = self.tokenizer.convert_tokens_to_ids([data[2*index+1].strip()])
      ans = [torch.tensor(min(vocab_size-1, ans[0]))]
      dct = {
          'ques': ques,
          'ques_str': line,
          'ans_str': data[2*index+1],
          'ans': ans,
          'img_file_name': img
      }

      if len(dct['ans']) == 1:
        self.data.append(dct)

  def __len__(self):
    return len(self.data) // 40

  def __getitem__(self, idx):
    dct = self.data[idx]
    img = Image.open(self.img_dir + dct['img_file_name']).crop((0, 0, 448, 448))
    img = np.array(img, dtype=np.uint8) / 255
    img = np.moveaxis(img, -1, 0)
    dct['img'] = torch.from_numpy(img)
    return dct

# if __name__ == '__main__':
#   print(torch.__version__)

In [33]:
def pad_collate(batch):
    # print(f'fix: {[len(x["ques"]) for x in batch]}')
    ques = [torch.tensor(x['ques']) for x in batch]
    ques = pad_sequence(ques, batch_first=True)
    # batch['ques'] = ques
    # print(f'{ques.shape}, {batch.__len__()}')
    for idx, x in enumerate(ques):
        batch[idx]['ques'] = x
    return batch

In [34]:
dl = DataLoader(VQADataset(ques_file='/content/qa.37.raw.train.txt', image_dir='/content/nyu_depth_images/', tokenizer=tokenizer), batch_size=2, collate_fn=pad_collate)

Iterating over questions: 0it [00:00, ?it/s]

In [13]:
for dct in dl:
  print(dct[0])
#   print(torch.cat(dct['ques']))
  break

{'ques': tensor([ 101, 2054, 2003, 2006, 1996, 2157, 2217, 1997, 1996, 2304, 7026, 1998,
        2006, 1996, 2187, 2217, 1997, 1996, 2417, 3242, 1999, 1996, 3746, 2509,
        1029,  102]), 'ques_str': 'what is on the right side of the black telephone and on the left side of the red chair in the image3 ?\n', 'ans_str': 'desk\n', 'ans': [tensor(4624)], 'img_file_name': 'image3.png', 'img': tensor([[[0.3412, 0.3333, 0.3020,  ..., 0.6157, 0.5647, 0.5569],
         [0.3490, 0.3373, 0.3020,  ..., 0.6078, 0.5686, 0.5686],
         [0.3412, 0.3373, 0.3059,  ..., 0.6039, 0.5686, 0.5725],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2863, 0.2745, 0.2431,  ..., 0.4510, 0.4000, 0.3922],
         [0.2824, 0.2706, 0.2353,  ..., 0.4431, 0.4039, 0.4000],
         [0.2745, 0.2706, 0.2353,  ..., 0.4392, 0.4039, 0.4078],
      

In [ ]:
dl.__len__()

3876

In [ ]:
'how many chairs are there in the image1437 ?'.strip('?').strip(' ').split()

['how', 'many', 'chairs', 'are', 'there', 'in', 'the', 'image1437']

In [ ]:
! python3 dataset.py

1.9.0+cu102


In [ ]:
with open('/content/qa.37.raw.train.txt', 'r') as f:
  # print(f.readlines())
  data = f.readlines()
  for idx, line in enumerate(data[::2]):
    print(2 * idx, line, data[2*idx+1])

## Train

In [26]:
import torch.nn as nn
from torch.optim import SGD, Adam
import torch.nn.functional as F

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

model = SAN(feat_dim, vocab_size, embed_size, n_attention_stacks, hidden_dim_img)
model = model.to(device)
criterian = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)
losses = []
print(device)

cuda


In [45]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [46]:
for epoch in range(10):
  current_loss = 0.0
  # get batches
  for batch in tqdm(dl):
    optimizer.zero_grad()

    # img = batch['img'].float().to(device)
    img = [x['img'] for x in batch]
    img = torch.stack(img, 0).float().to(device)
    # print()

    # ques = torch.cat(batch['ques']).to(device)
    ques = [x['ques'] for x in batch]
    # print(len(ques), ques[0].shape)
    ques = torch.stack(ques).to(device)

    # ans = torch.cat(batch['ans']).to(device)
    ans = [x['ans'][0] for x in batch]
    ans = torch.tensor(ans).to(device)

    # print(f'Ques: {ques.shape}, {ques}')
    out = model(img, ques)
    # print(f'Shapes: img: {img.shape},\n ques: {ques.shape}: {ques},\n ans: {ans.shape}: {ans},\n out; {out.shape}')

    loss = criterian(out, ans)
    # print(F.softmax(out, dim=0).detach().cpu().numpy(), ans.detach().cpu().numpy())

    loss.backward()
    optimizer.step()
    # print(loss)

    current_loss += loss.item() * len(batch)
    # print(f'Cur Loss: {current_loss}')

  print(f'Epoch: {epoch}, Loss: {current_loss:.2f}')

  0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 0, Loss: 651.57


  0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 1, Loss: 311.04


  0%|          | 0/48 [00:00<?, ?it/s]

...

Epoch: 7, Loss: 289.90


  0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 8, Loss: 289.71


  0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 9, Loss: 289.55


In [124]:
torch.cuda.is_available()

True

In [15]:
model(
    torch.rand((1, 3, 448, 448)).cuda(),
    torch.randint(0, 10, (1, 6)).cuda()
)

Img Rep: tensor([[[-6.6765e+25, -6.7470e+25, -6.6527e+25,  ..., -6.4934e+25,
          -6.8276e+25, -6.6186e+25],
         [-1.5914e+25, -1.5455e+25, -1.5575e+25,  ..., -1.5390e+25,
          -1.6632e+25, -1.5574e+25],
         [-1.1505e+25, -1.1366e+25, -1.1662e+25,  ..., -1.1309e+25,
          -1.2167e+25, -1.1706e+25],
         ...,
         [-2.9615e+25, -3.0396e+25, -2.8773e+25,  ..., -2.9591e+25,
          -2.9267e+25, -2.9280e+25],
         [-6.5078e+25, -6.5701e+25, -6.5172e+25,  ..., -6.3876e+25,
          -6.7787e+25, -6.3743e+25],
         [ 2.7307e+01,  2.7340e+01,  2.7280e+01,  ...,  2.7359e+01,
           2.7279e+01,  2.7343e+01]]], device='cuda:0', grad_fn=<AddBackward0>)
Ques Rep: tensor([[ 1.0000e+00,  0.0000e+00,  2.0000e+00,  0.0000e+00,  0.0000e+00,
          2.0000e+00,  4.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          0.0000e+00, -1.0000e+00,  2.0000e+00,  4.0000e+00,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00

tensor([[ 4.3755e+30,  3.1859e+30,  1.0416e+31,  ..., -1.8936e+31,
         -1.7635e+31, -2.5045e+30]], device='cuda:0', grad_fn=<AddmmBackward>)

#### Things Learnt:
- collate_fun & pad_sequence
- Model & input should be on the same device
- torch.cat & torch.stack
- Vocab & embed size for nn.Embeddings

###### Get the dataset

In [14]:
%%capture
! wget http://datasets.d2.mpi-inf.mpg.de/mateusz14visual-turing/nyu_depth_images.tar

In [15]:
%%capture
! tar -xvf nyu_depth_images.tar

In [16]:
! find . -name image3.png

./nyu_depth_images/image3.png


In [17]:
# Image.open('./nyu_depth_images/image3.png')

In [18]:
%%capture
! wget https://raw.githubusercontent.com/jayantk/lsp/master/data/daquar/reduced/qa.37.raw.train.txt

Rough

In [19]:
x_seq = [torch.tensor([5, 18, 29]), torch.tensor([32, 100]), torch.tensor([699, 6, 9, 17])]
x_padded = pad_sequence(x_seq, batch_first=True, padding_value=0)

In [20]:
x = torch.randn(2, 3)
print(f'{x.size()}')

# add more rows (thus increasing the dimensionality of the column space to 2 -> 6)
xnew_from_cat = torch.stack((x, x, x), 0)
print(f'{xnew_from_cat.size()}')

torch.Size([2, 3])
torch.Size([3, 2, 3])


In [21]:
out = torch.rand((2, 9000))
ans = torch.randint(0, 3, (2,))
out.shape, ans.shape

(torch.Size([2, 9000]), torch.Size([2]))

In [22]:
nn.CrossEntropyLoss()(out, ans)

tensor(9.2859)

In [23]:
x = torch.rand(10, 196, 1)

In [24]:
F.softmax(x, dim=1).sum(dim=1)

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]])